### 查詢擴充（Expand Query）

#### 方法說明
自動為原始查詢添加相關詞彙，包含同義詞、相關概念等，以提升檢索範圍。這個技術能夠彌補用戶查詢詞彙不足的問題，提高檢索的召回率。

#### 技術原理
- 使用詞彙擴展技術識別同義詞
- 利用知識圖譜找出相關概念
- 基於語義向量計算相似詞彙
- 結合領域知識進行概念擴展

#### 應用場景
- 提升檢索召回率
- 處理用戶查詢詞彙貧乏問題
- 跨語言檢索支援
- 專業術語擴展


In [1]:
import time
import os
import openai

class QueryExpander:
    def __init__(self):
        self.client = openai.OpenAI(
            api_key=os.environ["LITELLM_PROXY_API_KEY"],
            base_url=os.environ["LITELLM_PROXY_API_BASE"]
        )

    def expand_query(self, original_query: str, model: str) -> str:
        """
        使用 LLM 進行查詢擴充
        """
        prompt = f"""
        請為以下查詢提供 5 個相關的擴充詞彙：

        原始查詢：{original_query}

        請提供：
        1. 同義詞
        2. 相關概念
        3. 上下位詞
        4. 相關術語

        只回傳擴充詞彙，輸出格式為 Array List，不要包含其他說明。
        輸出例子：
        [
            "貓咪",
            "貓咪照片",
            "貓咪影像", 
            "貓咪圖片",
            "貓咪影片"
        ]
        """

        try:
            response = self.client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}]
            )
            
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            print(f"API 調用錯誤: {e}")
            return original_query

# 初始化查詢擴寫器
expander = QueryExpander()

#### Example 實作範例


In [9]:
original_queries = [
    "我想找關於胖橘貓咪的照片",
    "請幫我找一些關於機器學習的資料，專注在電腦視覺領域",
    "有沒有教怎麼做戚風蛋糕的影片？",
    "台北有什麼好玩的地方嗎？三天兩夜"
]

##### 範例：同義詞擴充

In [11]:
print("=== 查詢擴寫範例 ===")
for i, query in enumerate(original_queries, 1):
    print(f"\n範例 {i}:")
    print(f"原始查詢: {query}")
    
    rewritten = expander.expand_query(query, "gpt-4.1-nano")
    print(f"擴寫查詢: {rewritten}")
    

=== 查詢擴寫範例 ===

範例 1:
原始查詢: 我想找關於胖橘貓咪的照片
擴寫查詢: [
    "橘色貓",
    "橘貓照片",
    "貓咪品種",
    "寵物圖片",
    "萌系貓咪"
]

範例 2:
原始查詢: 請幫我找一些關於機器學習的資料，專注在電腦視覺領域
擴寫查詢: [
    "電腦視覺",
    "圖像辨識",
    "深度學習",
    "卷積神經網路",
    "影像處理"
]

範例 3:
原始查詢: 有沒有教怎麼做戚風蛋糕的影片？
擴寫查詢: [
    "戚風蛋糕制作方法",
    "戚風蛋糕食譜",
    "戚風蛋糕製作影片",
    "如何做戚風蛋糕",
    "海綿蛋糕影片"
]

範例 4:
原始查詢: 台北有什麼好玩的地方嗎？三天兩夜
擴寫查詢: [
    "台北景點",
    "旅遊行程",
    "文化古蹟",
    "住宿推薦",
    "觀光景點"
]


### 查詢重寫（Query Rewrite）

##### 方法說明
自動優化使用者的原始查詢語句，使其更符合檢索系統的特性。這個技術能夠將自然語言查詢轉換為更精確的檢索詞彙，提升檢索效果。

##### 技術原理
- 使用 LLM 分析原始查詢的語意
- 識別關鍵詞並進行優化
- 去除冗餘詞彙，強化檢索核心概念
- 適應不同檢索系統的特性

##### 應用場景
- 語音轉文字查詢優化
- 口語化查詢正規化
- 多語言查詢統一
- 檢索系統適配


In [16]:
import time
import os
import openai

class QueryRewriter:
    def __init__(self):
        self.client = openai.OpenAI(
            api_key=os.environ["LITELLM_PROXY_API_KEY"],
            base_url=os.environ["LITELLM_PROXY_API_BASE"]
        )

    def rewrite_query(self, original_query: str, model: str) -> str:
        """
        重寫查詢語句
        
        Args:
            original_query: 原始查詢語句
            model: 使用的模型名稱 (groq/llama3-8b-8192 或 openai/gpt-4.1-nano-2025-04-14)
            
        Returns:
            重寫後的查詢語句
        """
        prompt = f"""
        你是一個專業的查詢優化助手。請將以下原始查詢重寫得更詳細。

        原始查詢：{original_query}

        只回傳重寫後的文本，不要包含其他說明和多餘的文字或符號，必須維持原來的格式且必須為句子、且不要改變句字的文法，像要維找問句
        輸出必須是正體中文。
        輸出例子：
        原始查詢: 我想找關於貓咪的照片，最好是胖橘
        重寫查詢: 我想找一些關於貓咪的照片，特別是那些身材較胖、毛色為橘色的貓咪，可以包含不同角度和姿勢的圖片。
        """

        try:
            response = self.client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}]
            )
            
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            print(f"API 調用錯誤: {e}")
            return original_query
    
# 初始化查詢重寫器
rewriter = QueryRewriter()

#### Example 實作範例


In [14]:
original_queries = [
    # "我想找關於胖橘貓咪的照片",
    # "請幫我找一些關於機器學習的資料，專注在電腦視覺領域",
    # "有沒有教怎麼做戚風蛋糕的影片？",
    "請問醫師子宮肌瘤術後的飲食需要注意什麼"
]

##### 範例 1：口語化查詢優化

In [14]:
print("=== 查詢重寫範例 ===")
for i, query in enumerate(original_queries, 1):
    print(f"\n範例 {i}:")
    print(f"原始查詢: {query}")
    
    rewritten = rewriter.rewrite_query(query, "gpt-4.1-nano")
    print(f"重寫查詢: {rewritten}")
    

=== 查詢重寫範例 ===

範例 1:
原始查詢: 我想找關於胖橘貓咪的照片
重寫查詢: 我想找一些關於胖橘色貓咪的照片，這些照片應該展示貓咪的不同姿勢、角度，並且是高品質的圖片，讓我能夠欣賞到牠們可愛的模樣。

範例 2:
原始查詢: 請幫我找一些關於機器學習的資料，專注在電腦視覺領域
重寫查詢: 請幫我搜集有關機器學習的相關資料，並且重點關注於電腦視覺領域的最新研究、技術應用，以及相關的理論與算法，包含學術論文、技術報告、行業應用案例以及開源工具或資源，並提供相關的學習資源或推薦的資料來源。

範例 3:
原始查詢: 有沒有教怎麼做戚風蛋糕的影片？
重寫查詢: 有沒有詳細教學如何製作戚風蛋糕的影片，內容包括所有材料準備、步驟示範以及烘焙技巧的教程影片？

範例 4:
原始查詢: 台北有什麼好玩的地方嗎？三天兩夜
重寫查詢: 台北有哪些必訪的景點和活動適合三天兩夜的行程安排，可以包含文化古蹟、市集、美食、夜景、博物館，以及適合家庭或情侶的休閒娛樂場所，並提供詳細的行程建議和推薦。


##### 比較使用 Groq vs. OpenAI GPT-4o-mini vs. Ollama Qwen3 0.6B 的用時

In [17]:
def compare_models(original_query_list: list, model_name: str):
    """
    比較不同模型的重寫效果
    """
    # 初始化查詢重寫器
    rewriter = QueryRewriter()
    start_time = time.time()
    print(f"==使用 {model_name} 重寫查詢 ==")
    for original_query in original_query_list:
        print(f"原始查詢: {original_query}")
        rewritten = rewriter.rewrite_query(original_query, model_name)
        print(f"重寫查詢: {rewritten}\n")
    end_time = time.time()
    print(f"重寫時間: {end_time - start_time} 秒\n")

# openai-gpt-4.1-nano-2025-04-14
# compare_models(original_queries, "gpt-4.1-nano")

# groq/llama-3.1-8b-instant
compare_models(original_queries, "groq/llama-3.1-8b-instant")

# groq/llama-3.3-70b-versatile
compare_models(original_queries, "groq/llama-3.3-70b-versatile")

# gemini/gemini-2.0-flash-lite
# compare_models(original_queries, "gemini/gemini-2.0-flash-lite")

==使用 groq/llama-3.1-8b-instant 重寫查詢 ==
原始查詢: 請問醫師子宮肌瘤術後的飲食需要注意什麼
重寫查詢: 我想問的是，醫師子宮肌瘤術後，我需要注意的飲食問題有哪些。

重寫時間: 0.5938947200775146 秒

==使用 groq/llama-3.3-70b-versatile 重寫查詢 ==
原始查詢: 請問醫師子宮肌瘤術後的飲食需要注意什麼
重寫查詢: 請問醫師，子宮肌瘤手術後的患者在飲食方面需要注意哪些事項，以避免并發症、加速恢復及保持身體健康？

重寫時間: 0.6476960182189941 秒

